In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5c72e000 @  0x7f111b2142a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
0.4.0
True


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import csv
import math 
import random
import gzip
from scipy.stats import bernoulli
import torch
from sklearn import metrics

import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch.nn.functional as F


In [0]:
nummotif=16 #number of motifs to discover
bases='ACGT' #DNA bases
# basesRNA='ACGU'#RNA bases
batch_size=200 #fixed batch size 

In [0]:
def dinucshuffle(sequence):
    b=[sequence[i:i+2] for i in range(0, len(sequence), 2)]
    random.shuffle(b)
    d=''.join([str(x) for x in b])
    return d

In [0]:
class Chip():
    def __init__(self,filename):
        self.file = filename
            
    def openFile(self):
        data_all=[]
        pos_data=[]

        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')

            for row in reader:

                ## When using Embedding
                pos_data.append(row[2])
                data_all.append([row[2],[1]])
                data_all.append([dinucshuffle(row[2]),[0]])


#         random.shuffle(train_dataset)
        train_dataset=data_all

        size=int(len(train_dataset)/3)
        firstvalid=train_dataset[:size]
        secondvalid=train_dataset[size:size+size]
        thirdvalid=train_dataset[size+size:]
        firsttrain=secondvalid+thirdvalid
        secondtrain=firstvalid+thirdvalid
        thirdtrain=firstvalid+secondvalid
        return firsttrain,firstvalid,secondtrain,secondvalid,thirdtrain,thirdvalid,train_dataset,pos_data

In [0]:
# chipseq=Chip('/content/drive/My Drive/Deepbind/ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha_AC.seq.gz')
chipseq=Chip('/content/drive/My Drive/Colab Notebooks/Chip-seq/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz')
# chipseq=Chip('/content/drive/My Drive/Colab Notebooks/Chip-seq/USF1_HepG2_USF-1_HudsonAlpha_AC.seq.gz')


train1,valid1,train2,valid2,train3,valid3,alldataset,pos_data=chipseq.openFile()


## Create Embedding Vocabulary

In [0]:
!pip install --upgrade gensim
# imports needed and set up logging
import gzip
import gensim 
# import logging
import multiprocessing

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [0]:
def Gen_Words(pos_data,kmer_len,s):
    out=[]

    for i in pos_data:

        kmer_list=[]
        for j in range(0,(len(i)-kmer_len)+1,s):

              kmer_list.append(i[j:j+kmer_len])

        out.append(kmer_list)

    return out


In [0]:
document= Gen_Words(pos_data,5,2)

In [0]:

Embepochs=100
Embsize=50
model = gensim.models.Word2Vec (document, size=Embsize,workers=multiprocessing.cpu_count())
model.train(document,total_examples=len(document),epochs=Embepochs)
# print(model.wv.vectors)
model.save("/content/drive/My Drive/Colab Notebooks/Model"+str(Embepochs)+"epochs"+str(Embsize)+"EmbSize")

## Let's Create our GRU Model!

In [0]:
class chipseq_dataset(Dataset):
    """ Diabetes dataset."""

    def __init__(self,xy=None,model=None,kmer_len=5,stride=2):
      
        self.kmer_len= kmer_len
        self.stride= stride
        data=[el[0] for el in xy]
        words_doc= self.Gen_Words(data,self.kmer_len,self.stride)
#         print(words_doc[0])
        x_data=[self.convert_data_to_index(el,model.wv) for el in words_doc]
#         print(x_data[0])
       
        
        self.x_data=np.asarray(x_data,dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.LongTensor(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
      

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
      
    def Gen_Words(self,pos_data,kmer_len,s):
        out=[]
        
        for i in pos_data:

            kmer_list=[]
            for j in range(0,(len(i)-kmer_len)+1,s):

                  kmer_list.append(i[j:j+kmer_len])
                
            out.append(kmer_list)
            
        return out


    def convert_data_to_index(self, string_data, wv):
      index_data = []
      for word in string_data:
          if word in wv:
              index_data.append(wv.vocab[word].index)
      return index_data

In [0]:
kmer_len=5
stride=2

Embepochs=100
Embsize=50

model1 = gensim.models.Word2Vec.load("/content/drive/My Drive/Colab Notebooks/Model"+str(Embepochs)+"epochs"+str(Embsize)+"EmbSize")
train1_dataset=chipseq_dataset(train1,model1,kmer_len,stride)
train2_dataset=chipseq_dataset(train2,model1,kmer_len,stride)
train3_dataset=chipseq_dataset(train3,model1,kmer_len,stride)
valid1_dataset=chipseq_dataset(valid1,model1,kmer_len,stride)
valid2_dataset=chipseq_dataset(valid2,model1,kmer_len,stride)
valid3_dataset=chipseq_dataset(valid3,model1,kmer_len,stride)
alldataset_dataset=chipseq_dataset(alldataset,model1,kmer_len,stride)

In [0]:
batchSize=200

train_loader1 = DataLoader(dataset=train1_dataset,batch_size=batchSize,shuffle=True)
train_loader2 = DataLoader(dataset=train2_dataset,batch_size=batchSize,shuffle=True)
train_loader3 = DataLoader(dataset=train3_dataset,batch_size=batchSize,shuffle=True)
valid1_loader = DataLoader(dataset=valid1_dataset,batch_size=batchSize,shuffle=False)
valid2_loader = DataLoader(dataset=valid2_dataset,batch_size=batchSize,shuffle=False)
valid3_loader = DataLoader(dataset=valid3_dataset,batch_size=batchSize,shuffle=False)
alldataset_loader=DataLoader(dataset=alldataset_dataset,batch_size=batchSize,shuffle=True)

train_dataloader=[train_loader1,train_loader2,train_loader3]
valid_dataloader=[valid1_loader,valid2_loader,valid3_loader]


In [0]:


# input of shape(batch_size,inp_chan,iW)
class GRUNet(nn.Module):
    def __init__(self, hidden_size,dropprob,stdneu):
      
        super(GRUNet, self).__init__()
        
        model1 = gensim.models.Word2Vec.load("/content/drive/My Drive/Colab Notebooks/Model"+str(Embepochs)+"epochs"+str(Embsize)+"EmbSize")
        weights = torch.FloatTensor(model1.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights, freeze=False)
        # Get embeddings for index 1
#         input = torch.LongTensor([1,2])
#         print(input)
        
        
        self.hidden_size=hidden_size
        self.rnn = nn.GRU(Embsize, hidden_size, num_layers=1, bidirectional=True).to(device)
        self.wHidden = torch.normal(torch.zeros(2*hidden_size,1)).to(device)
        self.wHiddenBias = torch.randn(1).to(device)
        torch.nn.init.xavier_uniform(self.wHidden)
        self.wHidden.requires_grad=True
        self.wHiddenBias.requires_grad=True
        self.drop1=nn.Dropout(p=dropprob).to(device)
        self.FC2= nn.Linear(20,1,bias=True).to(device)
        
   
        for layer_p in self.rnn._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    # print(p, self.rnn.__getattr__(p))
                    nn.init.xavier_uniform(self.rnn.__getattr__(p))
       
    def forward(self, x):
    
    
    
#       input = torch.LongTensor(x)
      
      x_emb= self.embedding(x)
#       print(x_emb[1,1,:])
      x_emb=x_emb.permute(1,0,2)
      output, hn = self.rnn(x_emb)
      ## from (1, N, hidden) to (N, hidden)
#       rearranged = hn.view(hn.size()[1], hn.size(2))
      Normal_GRU=output[-1, :, :self.hidden_size]
      Rev_GRU=output[0, :, self.hidden_size:]
      Concat_GRU = torch.cat((Normal_GRU, Rev_GRU), 1)
      Concat_GRU=self.drop1(Concat_GRU)
      hid=Concat_GRU @ self.wHidden
      hid.add_(self.wHiddenBias)
#       inter=F.relu(hid)
#       inter=self.drop1(inter)
#       hid=self.FC2(inter)
      output = torch.sigmoid(hid)
      return output
      
      
     
        
    


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
# print(device)
learning_rate=0.01
momentum_rate=0.95
hidden_size=80
Embepochs=100
Embsize=50
dropprob=0.6
stdneu=1e-2

for CV in range(3):
  model = GRUNet(hidden_size,dropprob,stdneu).to(device)
#   optimizer = torch.optim.SGD([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate, momentum=momentum_rate,weight_decay=1e-5)
#   optimizer = torch.optim.Adam([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr = learning_rate, weight_decay=1e-5)
  optimizer= torch.optim.Adagrad([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate, weight_decay=1e-2)

  model.train()
  for iter in range(50):
    for i ,(data,target) in enumerate(train_dataloader[CV]):
#         print(data.shape)
#         data= data.permute(2,0,1)
        data = data.to(device)
        
        target = target.to(device)
        output = model(data)
        loss = F.binary_cross_entropy(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
  with torch.no_grad():
    model.eval()
    ## Train Auc
    auc=[]
    for i, (data, target) in enumerate(train_dataloader[CV]):
#         data= data.permute(2,0,1)
        data = data.to(device)
        target = target.to(device)
        
        # Forward pass
        output = model(data)
        pred=output.cpu().detach().numpy().reshape(output.shape[0])
        labels=target.cpu().numpy().reshape(output.shape[0])

        auc.append(metrics.roc_auc_score(labels, pred))

    print('AUC performance for training', np.mean(auc))
    
    
    auc=[]
    for i, (data, target) in enumerate(valid_dataloader[CV]):
#         data= data.permute(2,0,1)
        data = data.to(device)
        target = target.to(device)
        
        # Forward pass
        output = model(data)
        pred=output.cpu().detach().numpy().reshape(output.shape[0])
        labels=target.cpu().numpy().reshape(output.shape[0])

        auc.append(metrics.roc_auc_score(labels, pred))

    print('AUC performance ', np.mean(auc))
    
  print('        Fold Done!           ##########################################               ')


In [0]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
# print(device)
learning_rate=0.05
momentum_rate=0.95
hidden_size=80 #=50
Embepochs=100
Embsize=50
dropprob=0.6# =0.6

best_auc=0
for stdneu in [1e-3, 1e-4, 1e-5]:
  for hidden_size in [50,80,100]:
    for learning_rate in [0.05, 0.02,0.01, 0.001]:
      for weight_decay in [1e-2, 1e-3, 1e-10]:
          AUC_all=[]
          
          for CV in range(3):
            model = GRUNet(hidden_size,dropprob,stdneu).to(device)
          #   optimizer = torch.optim.SGD([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate, momentum=momentum_rate,weight_decay=1e-2)
          #   optimizer = torch.optim.Adam([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr = learning_rate, weight_decay=1e-2)
            optimizer= torch.optim.Adagrad([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate, weight_decay=weight_decay)

            model.train()
            for iter in range(50):
              for i ,(data,target) in enumerate(train_dataloader[CV]):
          #         print(data.shape)
          #         data= data.permute(2,0,1)
                  data = data.to(device)

                  target = target.to(device)
                  output = model(data)
                  loss = F.binary_cross_entropy(output,target)
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

            with torch.no_grad():
              model.eval()
              ## Train Auc
              auc=[]
              for i, (data, target) in enumerate(train_dataloader[CV]):
          #         data= data.permute(2,0,1)
                  data = data.to(device)
                  target = target.to(device)

                  # Forward pass
                  output = model(data)
                  pred=output.cpu().detach().numpy().reshape(output.shape[0])
                  labels=target.cpu().numpy().reshape(output.shape[0])

                  auc.append(metrics.roc_auc_score(labels, pred))

              print('AUC performance for training', np.mean(auc))


              auc=[]
              for i, (data, target) in enumerate(valid_dataloader[CV]):
          #         data= data.permute(2,0,1)
                  data = data.to(device)
                  target = target.to(device)

                  # Forward pass
                  output = model(data)
                  pred=output.cpu().detach().numpy().reshape(output.shape[0])
                  labels=target.cpu().numpy().reshape(output.shape[0])

                  auc.append(metrics.roc_auc_score(labels, pred))

              print('AUC performance ', np.mean(auc))
              AUC_all.append(np.mean(auc))
            print('   #####     Fold Done!           ##########################################               ')
          print( '====================================  ONE CROSS VALIDATION DONE !! ===========================================')
          if best_auc < np.mean(AUC_all):
            best_auc=np.mean(AUC_all)
            best_hidden_size=hidden_size
            best_learning_rate=learning_rate
            best_weight_decay=weight_decay
            print('================================= Best Parametyers So Far!!! ========================================', best_auc,best_hidden_size,best_learning_rate,best_weight_decay)            

print('Finally!!!! best AUC:   ', best_auc, '   best parameters:   ', best_hidden_size,best_learning_rate,best_weight_decay)

AUC performance for training 0.896127696977188
AUC performance  0.854650505050505
   #####     Fold Done!           ##########################################               
AUC performance for training 0.9209047125924446
AUC performance  0.8259654188948308
   #####     Fold Done!           ##########################################               
AUC performance for training 0.9303222532668196
AUC performance  0.7870200435729846
   #####     Fold Done!           ##########################################               
====================================  ONE CROSS VALIDATION DONE !! ===========================================
================================= Best Parametyers So Far!!! ======================================== 0.8225453225061067 50 0.05 0.01
AUC performance for training 0.9948282620315934
AUC performance  0.8219837195484254
   #####     Fold Done!           ##########################################               
AUC performance for training 0.9995064964413002
AUC p

### Testing Phase! (Training on all Training data)

In [0]:
class Chip_test():
    def __init__(self,filename):
        self.file = filename
            
    def openFile(self):
        data_all=[]
        pos_data=[]

        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')

            for row in reader:

                ## When using Embedding
                data_all.append([row[2],[int(row[3])]])


#         random.shuffle(train_dataset)
        train_dataset=data_all


        return train_dataset

In [0]:
chipseq_test=Chip_test('/content/drive/My Drive/Colab Notebooks/Chip-seq/ELK1_GM12878_ELK1_(1277-1)_Stanford_B.seq.gz')
# chipseq_test=Chip_test('/content/drive/My Drive/Colab Notebooks/Chip-seq/USF1_HepG2_USF-1_HudsonAlpha_B.seq.gz')
test_data=chipseq_test.openFile()
test_dataset=chipseq_dataset(test_data,model1,kmer_len,stride)
batchSize=test_dataset.__len__()
test_loader = DataLoader(dataset=test_dataset,batch_size=batchSize,shuffle=False)

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
# print(device)
learning_rate=0.02
momentum_rate=0.95
hidden_size=80
Embepochs=100
Embsize=50
dropprob=0.8
stdneu=1e-5
lr_dec= learning_rate / 200

model = GRUNet(hidden_size,dropprob,stdneu).to(device)
optimizer = torch.optim.SGD([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate, momentum=momentum_rate,weight_decay=1e-2)
# optimizer = torch.optim.Adam([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr = learning_rate, weight_decay=1e-5)
# optimizer= torch.optim.Adagrad([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate,lr_decay=lr_dec, weight_decay=1e-2)


for iter in range(200):
  model.train()
  for i ,(data,target) in enumerate(alldataset_loader):
#         print(data.shape)
#         data= data.permute(2,0,1)
      data = data.to(device)

      target = target.to(device)
      output = model(data)
      loss = F.binary_cross_entropy(output,target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  with torch.no_grad():
    model.eval()
    ## Train Auc
    auc=[]
    for i, (data, target) in enumerate(alldataset_loader):
  #         data= data.permute(2,0,1)
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        output = model(data)
        pred=output.cpu().detach().numpy().reshape(output.shape[0])
        labels=target.cpu().numpy().reshape(output.shape[0])

        auc.append(metrics.roc_auc_score(labels, pred))

    print('AUC performance for training', np.mean(auc))
     

  

print('       Done!           ##########################################               ')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


AUC performance for training 0.5398152180407366
AUC performance for training 0.5863425384973051
AUC performance for training 0.6238028635988235
AUC performance for training 0.6563810314145438
AUC performance for training 0.6517396933793037
AUC performance for training 0.6444054686747706


KeyboardInterrupt: ignored

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
# print(device)
learning_rate=0.03
momentum_rate=0.95
hidden_size=80
Embepochs=100
Embsize=50
dropprob=0.5
stdneu=1e-3
lr_dec= learning_rate / 400

model = GRUNet(hidden_size,dropprob,stdneu).to(device)
# optimizer = torch.optim.SGD([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate, momentum=momentum_rate,weight_decay=1e-2)
# optimizer = torch.optim.Adam([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr = learning_rate, weight_decay=1e-5)
optimizer= torch.optim.Adagrad([model.wHidden,model.wHiddenBias]+[param for param in model.parameters()], lr=learning_rate,lr_decay=lr_dec, weight_decay=0.015)


for iter in range(400):
  model.train()
  for i ,(data,target) in enumerate(alldataset_loader):
#         print(data.shape)
#         data= data.permute(2,0,1)
      data = data.to(device)

      target = target.to(device)
      output = model(data)
      loss = F.binary_cross_entropy(output,target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  with torch.no_grad():
    model.eval()
    ## Train Auc
    auc=[]
    for i, (data, target) in enumerate(alldataset_loader):
  #         data= data.permute(2,0,1)
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        output = model(data)
        pred=output.cpu().detach().numpy().reshape(output.shape[0])
        labels=target.cpu().numpy().reshape(output.shape[0])

        auc.append(metrics.roc_auc_score(labels, pred))

    print('AUC performance for training', np.mean(auc))
    
  with torch.no_grad():
      model.eval()
      auc=[]
     
      for i, (data, target) in enumerate(test_loader):
          data = data.to(device)
          target = target.to(device)

          # Forward pass
          output = model(data)

          pred=output.cpu().detach().numpy().reshape(output.shape[0])
          labels=target.cpu().numpy().reshape(output.shape[0])
          
          auc.append(metrics.roc_auc_score(labels, pred))
  #                         
      AUC_training=np.mean(auc)
      print('AUC on test data = ',AUC_training)


  

print('       Done!           ##########################################               ')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


AUC performance for training 0.6939058025158655
AUC on test data =  0.5670040000000001
AUC performance for training 0.6906519796410481
AUC on test data =  0.562784
AUC performance for training 0.6962773019637154
AUC on test data =  0.583256
AUC performance for training 0.6969872272903005
AUC on test data =  0.563428
AUC performance for training 0.6975014479019772
AUC on test data =  0.5830879999999999
AUC performance for training 0.7073679073824798
AUC on test data =  0.578152
AUC performance for training 0.7083424901371416
AUC on test data =  0.58384
AUC performance for training 0.7114190640267811
AUC on test data =  0.588854
AUC performance for training 0.7159469868978066
AUC on test data =  0.585056
AUC performance for training 0.7167401132296574
AUC on test data =  0.588824
AUC performance for training 0.7196249226240178
AUC on test data =  0.58388
AUC performance for training 0.7220145530851181
AUC on test data =  0.592484
AUC performance for training 0.7224499260450722
AUC on tes

In [0]:
with torch.no_grad():
      model.eval()
      auc=[]
     
      for i, (data, target) in enumerate(test_loader):
          data = data.to(device)
          target = target.to(device)

          # Forward pass
          output = model(data)

          pred=output.cpu().detach().numpy().reshape(output.shape[0])
          labels=target.cpu().numpy().reshape(output.shape[0])
          
          auc.append(metrics.roc_auc_score(labels, pred))
  #                         
      AUC_training=np.mean(auc)
      print('AUC on test data = ',AUC_training)

AUC on test data =  0.9066879999999999
